In [1]:
from IPython.display import display
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

In [3]:

#We first Specify the file path to the FASTA file
fasta_file_path = "XM_044419112.1[1..663].fa"
try:
    #Read the FASTA file using SeqIO.read
    record = SeqIO.read(fasta_file_path, "fasta")
    #Print the sequence record and its length
    print(str(record) + "\n")
    rec_seq = record.seq
    len_seq = print("Sequence Length = " + str(len(rec_seq)))
except FileNotFoundError:
    print(f"File not found: {fasta_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


ID: ref|XM_044419112.1|:1-663
Name: ref|XM_044419112.1|:1-663
Description: ref|XM_044419112.1|:1-663 PREDICTED: Varanus komodoensis neurturin (NRTN), transcript variant X1, mRNA
Number of features: 0
Seq('ATGAAGGTATGGAAGTTTGCAGCCATCGCCTCCATGGTTTTCAGTTCCATGCTG...TGA')

Sequence Length = 663


##### 2. Calculate Simple Statistics of each nitrogen base (C, G, A or T) over the total base pair.

In [4]:
def DNAFrequency(A, C, G, T):
    count_A = 0
    count_C = 0
    count_G = 0
    count_T = 0
    noise = 0
    #iterate through each character
    for char in rec_seq:
        if char == A:
            count_A += 1
        elif char == C:
            count_C += 1
        elif char == G:
            count_G += 1
        elif char == T:
            count_T += 1
        elif char != A or C or G or T:
            noise += 1
    
    total_A = print("Total DNA A = "+ str(count_A))
    total_C = print("Total DNA C = "+ str(count_C))
    total_G = print("Total DNA G = "+ str(count_G))
    total_T = print("Total DNA T = "+ str(count_T))
    total_noise = print("Total Noise/Error = "+ str(noise))

    print("\nTotal Frequency\n")

    print("Frequency A = " + str(count_A/(len(rec_seq)-noise)))
    print("Frequency C = " + str(count_C/(len(rec_seq)-noise)))
    print("Frequency G = " + str(count_G/(len(rec_seq)-noise)))
    print("Frequency T = " + str(count_T/(len(rec_seq)-noise)))

 
#call the dna frequency function
DNAFrequency('A', 'C', 'G', 'T')


Total DNA A = 132
Total DNA C = 198
Total DNA G = 202
Total DNA T = 131
Total Noise/Error = 0

Total Frequency

Frequency A = 0.19909502262443438
Frequency C = 0.2986425339366516
Frequency G = 0.3046757164404223
Frequency T = 0.1975867269984917


##### Develop a sub program to do moving window for calculating frequency of each base nitrogen, to observe the unusual motif. Width of moving window can be decided by experimental approach to observe possibility of finding unusual motif at specific location. 

In [5]:
#the recoreded dna sequence
rec_seq = '''ATGAAGGTATGGAAGTTTGCAGCCATCGCCTCCATGGTTTTCAGTTCCATGCTGTCCATGTTGGTTTGCA
GAGACCTGTTTGGAGGGAGCCCTGCATCTACCTTCTTCCCCTCAGCACTGTTCCCGAGAGCATCCTCACC
GCCCCCTTGGCTCGCCGATGGTCAGAAGAGGCACCCGCGGGATCTGGAGCGCCGCAGCTCCCTGATCACC
CAGTACAGCACGTTGTTCGAGAGCTACACTGAGGGAGAGATCCGGCAGCTCATCTCCACACTGATTGAGA
GGTACAGCCAGTCCATGAACTCCGGGGGCCACGAACTGCCGCTCTTGGAGCGCGTGGGCACCCGGATGAA
GCGTGCCAAGGCCCGGCACAAGCCCTGCTCCCTCAAGGAGCTGGAGGTGACGGTCAGCGAGCTGGGCCTG
GGCTACAAGTCCGACGAGACTGTCTTGTTCCGCTACTGCAGCGGCACCTGTGACTCGGCTGTGCGGAACT
ACGACCTCTCCTTGAAGAATGTCCGGAGCATGCGGAAGATCAAGAAGGAGAAGGTGCGAGCCCGTCCGTG
CTGCCGGCCGCTCTCGTACGACGACGACGTTTCCTTCCTGGACACGGGAAACCGGTACTACACCGTGAAC
GAGGTGTCGGCAAAGGAGTGTGGCTGCGTGTGA'''


def calculate_base_frequencies(seq, window_size):
    base_frequencies = {
        'A': [],
        'C': [],
        'G': [],
        'T': []
    }

    for i in range(len(seq) - window_size + 1):
        window = seq[i:i + window_size]
        window_length = len(window)

        # Calculate frequencies of A, C, G, and T in the window
        base_frequencies['A'].append(window.count('A') / window_length)
        base_frequencies['C'].append(window.count('C') / window_length)
        base_frequencies['G'].append(window.count('G') / window_length)
        base_frequencies['T'].append(window.count('T') / window_length)

    return base_frequencies

# Define a function to calculate AT-GC density
def calculate_at_gc_density(seq, window_size):
    at_density = []
    gc_density = []

    for i in range(len(seq) - window_size + 1):
        window = seq[i:i + window_size]
        at_count = window.count('A') + window.count('T')
        gc_count = window.count('G') + window.count('C')
        window_length = len(window)

        at_density.append(at_count / window_length)
        gc_density.append(gc_count / window_length)

    return at_density, gc_density

# Define a function to plot base frequencies and AT-GC density
def plot_base_frequencies_and_at_gc_density(window_size):
    base_frequencies = calculate_base_frequencies(rec_seq, window_size)
    at_density, gc_density = calculate_at_gc_density(rec_seq, window_size)
    x = np.arange(len(base_frequencies['A']))
    # Create subplots for base frequencies and AT-GC density
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    fig.suptitle("Base Frequencies and AT-GC Density in Moving Window")
    ax1.set_ylabel("Frequency")
    ax1.plot(x, base_frequencies['A'], label='A', color='blue')
    ax1.plot(x, base_frequencies['C'], label='C', color='green')
    ax1.plot(x, base_frequencies['G'], label='G', color='red')
    ax1.plot(x, base_frequencies['T'], label='T', color='purple')
    ax1.legend()
    ax1.grid()
    ax2.set_xlabel("Window Index")
    ax2.set_ylabel("Density")
    ax2.plot(x, at_density, label='AT Density', color='blue')
    ax2.plot(x, gc_density, label='GC Density', color='green')
    ax2.legend()
    ax2.grid()
    plt.tight_layout()
    plt.show()

# Create an interactive widget to adjust the window size
window_size_widget = widgets.IntSlider(value=1, min=1, max=len(rec_seq), description='Window Size:')
widgets.interactive(plot_base_frequencies_and_at_gc_density, window_size=window_size_widget)

interactive(children=(IntSlider(value=1, description='Window Size:', max=672, min=1), Output()), _dom_classes=…

##### Do the process for Dimer (CG, AT)

In [7]:
def dimer():
    AA, AT, AG, AC = 0,0,0,0
    TA, TT, TG, TC = 0,0,0,0
    GA, GT, GG, GC = 0,0,0,0
    CA, CT, CG, CC = 0,0,0,0
    for i in range((len(rec_seq)//2)-1):
        if rec_seq[i*2] == 'A':
            if rec_seq[i*2+1] == 'A':
                AA +=1
            elif rec_seq[i*2+1] == 'T':
                AT +=1
            elif rec_seq[i*2+1] == 'G':
                AG +=1
            elif rec_seq[i*2+1] == 'C':
                AC +=1
        if rec_seq[i*2] == 'T':
            if rec_seq[i*2+1] == 'A':
                TA += 1
            elif rec_seq[i*2+1] == 'T':
                TT += 1
            elif rec_seq[i*2+1] == 'G':
                TG += 1
            elif rec_seq[i*2+1] == 'C':
                TC += 1
        if rec_seq[i*2] == 'G':
            if rec_seq[i*2+1] == 'A':
                GA += 1
            elif rec_seq[i*2+1] == 'T':
                GT += 1
            elif rec_seq[i*2+1] == 'G':
                GG += 1
            elif rec_seq[i*2+1] == 'C':
                GC += 1
        if rec_seq[i*2] == 'C':
            if rec_seq[i*2+1] == 'A':
                CA += 1
            elif rec_seq[i*2+1] == 'T':
                CT += 1
            elif rec_seq[i*2+1] == 'G':
                CG += 1
            elif rec_seq[i*2+1] == 'C':
                CC += 1
    
    dAA, dAT, dAG, dAC = AA/(len(rec_seq)/2), AT/(len(rec_seq)/2), AG/(len(rec_seq)/2), AC/(len(rec_seq)/2)
    dTA, dTT, dTG, dTC = TA/(len(rec_seq)/2), TT/(len(rec_seq)/2), TG/(len(rec_seq)/2), TC/(len(rec_seq)/2)
    dGA, dGT, dGG, dGC = GA/(len(rec_seq)/2), GT/(len(rec_seq)/2), GG/(len(rec_seq)/2), GC/(len(rec_seq)/2)
    dCA, dCT, dCG, dCC = CA/(len(rec_seq)/2), CT/(len(rec_seq)/2), CG/(len(rec_seq)/2), CC/(len(rec_seq)/2)

    print("2-Mer Frequencies")
    print("AA : " + str(dAA) + "\tAC : " + str(dAC)+"\tAG : "+str(dAG)+"\tAT : "+str(dAT))
    print("CA : " + str(dCA) + "\tCC : " + str(dCC)+"\tCG : "+str(dCG)+"\tAT : "+str(dCT))
    print("GA : " + str(dGA) + "\tGC : " + str(dGC)+"\tGG : "+str(dGG)+"\tGT : "+str(dGT))
    print("TA : " + str(dTA) + "\tTC : " + str(dTC)+"\tTG : "+str(dTG)+"\tTT : "+str(dTT))

dimer()


2-Mer Frequencies
AA : 0.03273809523809524	AC : 0.07142857142857142	AG : 0.07738095238095238	AT : 0.03571428571428571
CA : 0.050595238095238096	CC : 0.09523809523809523	CG : 0.06845238095238096	AT : 0.06547619047619048
GA : 0.07142857142857142	GC : 0.08333333333333333	GG : 0.07142857142857142	GT : 0.06845238095238096
TA : 0.011904761904761904	TC : 0.05357142857142857	TG : 0.07142857142857142	TT : 0.041666666666666664
